# Lisbon, Portugal

I have chosen to explore the Lisbon, Portugal database as this is the homeland of my parents and I have just vacationed there for the first time in two decades. Seeing how the data pans to my own ancedotal evidence from my trip will be interesting.

Data: https://mapzen.com/data/metro-extracts/metro/lisbon_portugal/

## Importing Data and Running the Count Tags Function
In this next cell, we will upload the Lisbon, Portugal OSM file and explore the data to determine the types of tags and the count of the tags therein. 

In [1]:
#!/usr/bin/env python
#-*- coding: utf-8 -*-
import xml.etree.cElementTree as ET
import pprint
import re
from collections import defaultdict
import csv
import codecs
import re
import xml.etree.cElementTree as ET
import cerberus
import schema

filename = "lisbon_portugal.osm"

#Exploring the data to determine the types of tags and the number of those tags. 

def count_tags(filename):
    tag_types = set()
    tag_dict = {}
    file = ET.iterparse(filename) 
    for event, elem in file:
        value = elem.tag
        if value not in tag_types:
            tag_types.add(value)
            tag_dict[value] = 1
        elif value in tag_types:
            tag_dict[value] += 1
    return tag_dict               




In [11]:
print (count_tags(filename))

{'node': 2007823, 'nd': 2522161, 'bounds': 1, 'member': 39247, 'tag': 698485, 'relation': 3994, 'way': 265109, 'osm': 1}


### Results
The above shows there to be 2,007,823 nodes, 265,109 ways, and 3,994 relations in the dataset. This is a good indication of the size of the dataset we will be working with. 

## Unique Contributors

Now, we will examine the contributors from the dataset for nodes and ways to determine how many unique contributors have contributed to the nodes and ways. We will be doing this using a custom function called node_way_contributors.

In [2]:
def node_way_contributors(filename):
    contributors = []
    unique_contributors = set()
    file = ET.iterparse(filename)
    for event,elem in file: 
        if elem.tag == "node" or elem.tag == "way":
            elem.get('user')
            contributors.append(elem.get('user'))
            unique_contributors.add(elem.get('user'))
    
    print (len(contributors))
    print (len(unique_contributors))
            

In [4]:
node_way_contributors(filename)

2272932
2817


### Results
We now know that 2,817 contributors have contributed data to our nodes and our ways, the two areas that we will be further exploring in this analysis

## Further Exploration of K
Using a function worked on in the case study example for Chicago, Illinois, we have used the key_type and process_map function to understand the actual makeup of the data set, particularly 'k' value sets with colons as we will be separating those in the future. 


In [26]:
lower = re.compile(r'^([a-z]|_)*$')
lower_colon = re.compile(r'^([a-z]|_)*:([a-z]|_)*$')
problemchars = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')


def key_type(element, keys):
    if element.tag == "tag":
        k = element.get('k')
        lower_result = bool(re.search(lower,k))
        lower_colon_result = bool(re.search (lower_colon,k))
        problemchars_result = bool(re.search (problemchars,k))
        if lower_result == True: 
            keys['lower'] += 1
        if lower_colon_result == True: 
            keys['lower_colon'] += 1
        if problemchars_result == True: 
            keys['problemchars'] += 1
            print k
        if lower_result  == False:
            if lower_colon_result == False: 
                if problemchars_result == False: 
                    keys['other'] +=1
           
    return keys



def process_map(filename):
    keys = {"lower": 0, "lower_colon": 0, "problemchars": 0, "other": 0}
    for _, element in ET.iterparse(filename):
        keys = key_type(element, keys)

    return keys

In [27]:
process_map(filename)

open air


{'lower': 604240, 'lower_colon': 90746, 'other': 3498, 'problemchars': 1}

### Results
The data shows 90,746 datapoints with a colon and only 1 that includes problem characters. After printing the problemchars result, we realized that it is only a string with a space inbetween two words ('open air'), which should not be an issue in further analysis. 

## What types of values to we have in K
Wanting to take a cursory look at the k-values after finding only 1 problemchars, we further examined the k values using a custom function "view_k_values" that takes in the file and returns a dictionary of unique k values and their count in the database. 

In [4]:
k_values_set = []
def view_k_values(osmfile):
    k_values = {}
    osm_file = open(osmfile,"r")
    for event, elem in ET.iterparse(osm_file, events =("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                k_value = tag.attrib['k']
                if k_value not in k_values_set: 
                    k_values_set.append(k_value)
                    k_values[k_value] = 1
                else: 
                    k_values[k_value] +=1
    return k_values
                
                    

In [34]:
view_k_values(filename)

{'destination:symbol:forward': 4,
 'seamark:beacon_lateral:system': 1,
 'maxspeed': 5638,
 'psv:forward': 10,
 'chain': 1,
 'INF_TELEFONE': 1,
 'seamark:light:4:category': 1,
 'destination:forward': 32,
 'handrail:center': 5,
 'health_specialty:meditation': 1,
 'turn:lanes:forward': 143,
 'post_box:type': 4,
 'passage_type': 1,
 'retail': 1,
 'placement:forward': 7,
 'is_in': 60,
 'abandoned:man_made': 12,
 'seamark:light:2:character': 4,
 'seamark:type': 255,
 'pastelaria': 1,
 'archeologic': 1,
 'fenced': 1,
 'created_by': 6761,
 'contact:pinterest': 1,
 'heritage:since': 85,
 'charge': 1,
 'restriction': 8,
 'crossing:barrier': 6,
 'camera:direction': 1,
 'cod_act_econ': 5,
 'watermill:disused': 2,
 'fax': 158,
 'indoor': 9,
 'attribution': 1,
 'alt_name:de': 1,
 'social_facility': 25,
 'telecom': 1,
 'internet': 1,
 'wave': 2,
 'icao': 7,
 'traffic_signals:minimap': 4,
 'recycling:magazines': 11,
 'facebook': 249,
 'population:date': 1,
 'motor_vehicle': 1807,
 'bridge:ref': 6,
 's

In [35]:
print len(k_values_set)

1071


### Results
Results of running the view_k_values funciton shows there to be 1,071 different k types we may run analysis on in the future. 

## Audit Street Names
Similar to what we have done in the Chicago.OSM case study, I have altered the audit_street_names function to pertain to Lisbon Portugal streets, as in Lisbon the street "type" (Road, Street) will actually be the FIRST word in the string. For example, "John's Road" would be "Rua de John" in Portuguese. 

In this function, I took the first word after split for each 'v' if the 'k' was "addr:street", added that to a dictionary, then iterated through the file and each time a new "street name" type was added I would increase by 1. 

In [5]:
def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit_street_names(osmfile):
    street_names = {}
    osm_file = open(osmfile, "r")
    for event, elem in ET.iterparse(filename, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    first_word = tag.attrib['v'].split()[0]
                    if first_word not in street_names: 
                        street_names[first_word] = 1
                    else:
                        street_names[first_word] += 1
    osm_file.close()

    return street_names

In [11]:
st_dict = audit_street_names(filename)
for key, value in st_dict.items():
    print key,value

Areia 1
Terraços 1
Escadinhas 11
Paredão 2
Alameda 82
R. 20
Lote 1
Torres 1
Bairro 20
Traveça 1
Beco 24
Alto 18
Pàtio 2
Marechal 6
Recinto 1
Travessia 1
da 1
Impasse 3
Armazém 2
António 1
Paço 4
Largo 302
doca 1
Pátio 41
Vila 52
Ponta 1
Praceta 568
Caminho 21
53 1
E.N.379 1
Actriz 1
André 1
Rossio 1
Zona 1
Poço 4
Regueirão 1
Area 3
Malhoa 1
AV. 1
Herdade 2
Edifício 1
LARGO 1
N119 1
Shopping 1
Rua 13018
Praia 3
Cruz 1
Palma 1
EN9 2
EN 14
Qta. 1
Campo 58
rua 13
Oitavos 1
Plataforma 1
Percurso 1
Praça 386
Calcada 4
Lagoas 1
Calçadinha 3
Antonio 1
Parada 2
Sacadura 1
Traversa 1
Tapada 1
Cova 1
Calçada 273
Avenida 2407
Jardim 8
praceta 1
Parque 4
Gen. 1
PRAÇA 2
travessa 1
Jangada 1
calçada 1
avenida 2
creche 1
IC 1
Circular 11
Estrada 433
Praca 1
Casal 5
PARQUE 1
Centro 1
Av. 79
Modelo 1
Eixo 3
Visconde 1
Mercado 2
Cais 3
estrada 3
RUa 1
Pct. 1
Qt. 2
R 4
Eça 7
Cruzes 4
Esplanada 1
RUA 10
Praceia 1
RUY 1
Terreiro 1
Travessa 544
ua 1
Passeio 29
Azinhaga 20
Maio 1
Trindade 1
Quinta 15
Lugar 1


### Results
A few problems were encountered when running the audit_street_names function. Firstly, there are varying cases used. Sometimes, roads are written in all UPPER CASE and sometimes in lower case. There also isn't a specific means of writing abbreviations. I will be using the data audited in this capture to refine a mapping dictionary to standardize some of the street name results. 

In addition, I would like to point out the function was written to take the first word from the tag.attrib['v'] as in Portuguese "road" or "street" would lead the sentence. 

## Changing Street Names
In the following function originally derived from the Case Study OSM, I customized the mapping dictionary to fit data audited in the audit function previously, ensuring that unique words and street types in the portuguese language are accurately changed. 

In [6]:
#street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)

mapping = { "AV.": "Avienda",
           "avienda" : "Avienda",
            "AV":"Avienda",
            "Av.":"Avienda",
           "Av" : "Avienda",
           "Calcada" : "Calçada",
           "calçada" : "Calçada",
           "E.N.379": "EN",
           "EN9" : "EN",
           "estrada": "Estrada",
           "ESTRADA" : "Estrada",
           "LARGO":"Largo",
           "PARQUE":"Parque",
           "PRAÇA": "Praça",
           "Praca":"Praça",
           'Pàtio':'Pátio',
           "Qt.":"Quinta",
           "Qta.":"Quinta",
           "R":"Rua",
           "R.":"Rua",
           "RUA":"Rua",
           "RUa":"Rua",
           "RUY": "Rua",
           'Rua\u200b':"Rua",  
           "rua" : "Rua",
          }


def audit_street_type(street_types, street_name):
    street_type = street_name.split()[0]
    street_types[street_type].add(street_name)


def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")


def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()

    return street_types


def update_name(name, mapping):
    words = name.split()
    sentence = ""
    for word in words:
        if word in mapping:
            word = mapping[word]
            sentence = sentence + word + " "
        else:
            if word == words[-1]:
                sentence = sentence + word
            else:
                sentence = sentence + word + " "
    return sentence


def street_names_updates():
    st_types = audit(filename)
    #pprint.pprint(dict(st_types))

    for st_type, ways in st_types.iteritems():
        for name in ways:
            better_name = update_name(name, mapping)
            print name, "=>", better_name.title()


In [8]:
street_names_updates()

Areia => Areia
Terraços da Ponte => Terraços Da Ponte
Escadinhas das Barrocas => Escadinhas Das Barrocas
Escadinhas da Fonte da Pipa => Escadinhas Da Fonte Da Pipa
Escadinhas Lord Byron => Escadinhas Lord Byron
Escadinhas Saúde => Escadinhas Saúde
Escadinhas do Espírito Santo da Pedreira => Escadinhas Do Espírito Santo Da Pedreira
Escadinhas da Praia => Escadinhas Da Praia
Escadinhas do Teixeira => Escadinhas Do Teixeira
Escadinhas do Duque => Escadinhas Do Duque
Escadinhas dos Torres => Escadinhas Dos Torres
Paredão => Paredão
Paredão Apoio => Paredão Apoio
Alameda da Beloura => Alameda Da Beloura
Alameda da Praia => Alameda Da Praia
Alameda Pocinho das Nascentes => Alameda Pocinho Das Nascentes
Alameda da Música => Alameda Da Música
Alameda Dom Afonso Henriques => Alameda Dom Afonso Henriques
Alameda das Linhas de Torres => Alameda Das Linhas De Torres
Alameda do Beato => Alameda Do Beato
Alameda de Santo António dos Capuchos => Alameda De Santo António Dos Capuchos
Alameda Doutor Jo

Rua de Santo António da Glória => Rua De Santo António Da Glória
Rua das Camélias => Rua Das Camélias
Rua do Viveiro => Rua Do Viveiro
Rua Amilcar Cabral => Rua Amilcar Cabral
Rua Professor Vieira de Almeida => Rua Professor Vieira De Almeida
Rua Dona Estefânia => Rua Dona Estefânia
Rua Júlio de Andrade => Rua Júlio De Andrade
Rua Jorge Amado => Rua Jorge Amado
Rua do Rouxinol => Rua Do Rouxinol
Rua das Margaridas => Rua Das Margaridas
Rua Bernardino Machado => Rua Bernardino Machado
Rua Cidade de Gabela => Rua Cidade De Gabela
Rua da Conceição da Glória => Rua Da Conceição Da Glória
Rua Fernando Vaz => Rua Fernando Vaz
Rua Primeiro de Maio => Rua Primeiro De Maio
Rua Dia Mundial da Criança => Rua Dia Mundial Da Criança
Rua da Quinta da Bolacha => Rua Da Quinta Da Bolacha
Rua de São Domingos => Rua De São Domingos
Rua João de Freitas Branco => Rua João De Freitas Branco
Rua de Palmela => Rua De Palmela
Rua Bulhão Pato => Rua Bulhão Pato
Rua Quinta da Varejeira => Rua Quinta Da Varejeir

Rua Venceslau de Morais => Rua Venceslau De Morais
Rua Teixeira de Pascoais => Rua Teixeira De Pascoais
Rua Doutor Gama Barros => Rua Doutor Gama Barros
Rua do Mercado => Rua Do Mercado
Rua da Artilharia 1 => Rua Da Artilharia 1
Rua do Lumiar => Rua Do Lumiar
Rua do Diário de Notícias => Rua Do Diário De Notícias
Rua dos Anjos => Rua Dos Anjos
Rua doutor Alberto Leite => Rua Doutor Alberto Leite
Rua Moinhos de Fanares => Rua Moinhos De Fanares
Rua da Margem => Rua Da Margem
Rua de Pedrouços => Rua De Pedrouços
Rua João Penha => Rua João Penha
Rua Prior do Crato => Rua Prior Do Crato
Rua de Macau => Rua De Macau
Rua Foros da Amora => Rua Foros Da Amora
Rua De Borges Carneiro;Rua Borges Carneiro => Rua De Borges Carneiro;Rua Borges Carneiro
Rua Parque Natural da Madeira => Rua Parque Natural Da Madeira
Rua Alto do Fandanguinho => Rua Alto Do Fandanguinho
Rua Lopes => Rua Lopes
Rua Andrade Corvo => Rua Andrade Corvo
Rua Cidade de Belo Horizonte => Rua Cidade De Belo Horizonte
Rua da Biolo

Praça Cidade do Luso => Praça Cidade Do Luso
Praça das Flores => Praça Das Flores
Praça Luis de Camóes => Praça Luis De Camóes
Praça 1 de Maio => Praça 1 De Maio
Praça D. Manuel I => Praça D. Manuel I
Praça da Figueira => Praça Da Figueira
Praça Engenheiro José Vaz Guedes => Praça Engenheiro José Vaz Guedes
Praça Dom Sebastião => Praça Dom Sebastião
Praça do Junqueiro => Praça Do Junqueiro
Praça José Fontana => Praça José Fontana
Praça Professor Rodriges Lapa => Praça Professor Rodriges Lapa
Praça Mário Azevedo Gomes => Praça Mário Azevedo Gomes
Praça Afonso de Albuquerque => Praça Afonso De Albuquerque
Praça dos Restauradores => Praça Dos Restauradores
Praça Doutor Francisco Sá Carneiro => Praça Doutor Francisco Sá Carneiro
Praça Ilha do Faial => Praça Ilha Do Faial
Praça do Chile => Praça Do Chile
Praça de São Paulo => Praça De São Paulo
Praça Marechal Humberto Delgado => Praça Marechal Humberto Delgado
Praça 25 ABRIL => Praça 25 Abril
Praça Natália Correia => Praça Natália Correia
P

Travessa Frei Fernão Rodrigues Monteiro => Travessa Frei Fernão Rodrigues Monteiro
Travessa dos Choupos => Travessa Dos Choupos
Travessa da Peixeira => Travessa Da Peixeira
Travessa das Amoreiras a Arroios => Travessa Das Amoreiras A Arroios
Travessa do Alcaide => Travessa Do Alcaide
Travessa do Terreiro do Trigo => Travessa Do Terreiro Do Trigo
Travessa da Memória => Travessa Da Memória
Travessa das Íris => Travessa Das Íris
Travessa Larga => Travessa Larga
Travessa do Convento das Bernardas => Travessa Do Convento Das Bernardas
Travessa José Augusto Saloio => Travessa José Augusto Saloio
Travessa Moinho da Torre => Travessa Moinho Da Torre
Travessa Júlio Brandão => Travessa Júlio Brandão
Travessa do Pereira => Travessa Do Pereira
Travessa das Flores => Travessa Das Flores
Travessa Sara Afonso => Travessa Sara Afonso
Travessa da Água-da-Flor => Travessa Da Água-Da-Flor
Travessa de Santa Teresa => Travessa De Santa Teresa
Travessa dos Ferreiros à Lapa => Travessa Dos Ferreiros À Lapa
T

## Writing to CSV
The following functions iterate over each element in the XML, shape those elements to fit SCHEMA established in a schema document, and writes those elements to indicated CSV files. 

In [102]:
OSM_PATH = "lisbon_portugal.osm"

NODES_PATH = "nodes.csv"
NODE_TAGS_PATH = "nodes_tags.csv"
WAYS_PATH = "ways.csv"
WAY_NODES_PATH = "ways_nodes.csv"
WAY_TAGS_PATH = "ways_tags.csv"

LOWER_COLON = re.compile(r'^([a-z]|_)+:([a-z]|_)+')
PROBLEMCHARS = re.compile(r'[=\+/&<>;\'"\?%#$@\,\. \t\r\n]')

SCHEMA = schema.schema

# Make sure the fields order in the csvs matches the column order in the sql table schema
NODE_FIELDS = ['id', 'lat', 'lon', 'user', 'uid', 'version', 'changeset', 'timestamp']
NODE_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_FIELDS = ['id', 'user', 'uid', 'version', 'changeset', 'timestamp']
WAY_TAGS_FIELDS = ['id', 'key', 'value', 'type']
WAY_NODES_FIELDS = ['id', 'node_id', 'position']

def shape_element(element, node_attr_fields=NODE_FIELDS, way_attr_fields=WAY_FIELDS,
                  problem_chars=PROBLEMCHARS, default_tag_type='regular'):
    """Clean and shape node or way XML element to Python dict"""

    node_attribs = {}
    way_attribs = {}
    way_nodes = []
    tags = []
    # Handle secondary tags the same way for both node and way elements

    if element.tag == 'node':
        
        #ESTABLISH THE NODE_ATTRIBS AS A DICTIONARY FOR EACH NODE
        
        for each in node_attr_fields: 
            node_attribs[each] = element.get(each)
        
        #ESTABLISH NODE TAGS AS DICTIONARIES PER SECONDARY TAG PER NODE
        for child in element:
            if child.tag == "tag":
                insert_tag = {}
                problemchars_test = PROBLEMCHARS.search(child.attrib['k'])
                lower_colon_test = LOWER_COLON.search(child.attrib['k'])
                if problemchars_test:
                    continue
                elif lower_colon_test:
                    insert_tag['id'] = element.attrib['id']
                    key = child.attrib['k'].split(":",1)[1]
                    type_value = child.attrib['k'].split(":",1)[0]
                    insert_tag['key'] = key
                    insert_tag['type'] = type_value
                    if child.attrib["k"] == 'addr:street':
                        insert_tag["value"] = update_name(child.attrib["v"], mapping)
                    else:
                        insert_tag['value'] = child.attrib['v']
                    tags.append(insert_tag)
                else:
                    insert_tag['id'] = element.attrib['id']
                    insert_tag['value'] = child.attrib['v']
                    insert_tag['key'] = child.attrib['k']
                    insert_tag['type'] = "regular"
                    tags.append(insert_tag)
        return {'node': node_attribs, 'node_tags': tags}
    
    elif element.tag == 'way':
        #ESTABLISH WAY_NODES
        position = 0
        for tag in element.iter("nd"):
            way_nodes.append({'id': element.get('id'), 'node_id': tag.get('ref'),'position': position})
            position += 1
        #ESTABLISH WAY_ATTRIBS
        for each in WAY_FIELDS: 
            way_attribs[each] = element.get(each)
        
        for child in element:
            if child.tag == "tag":
                insert_tag = {}
                problemchars_test = PROBLEMCHARS.search(child.attrib['k'])
                lower_colon_test = LOWER_COLON.search(child.attrib['k'])
                if problemchars_test:
                    continue
                elif lower_colon_test:
                    insert_tag['id'] = element.attrib['id']
                    key = child.attrib['k'].split(":",1)[1]
                    type_value = child.attrib['k'].split(":",1)[0]
                    insert_tag['key'] = key
                    insert_tag['type'] = type_value
                    if child.attrib["k"] == 'addr:street':
                        insert_tag["value"] = update_name(child.attrib["v"], mapping)
                    else:
                        insert_tag['value'] = child.attrib['v']
                    tags.append(insert_tag)
                else:
                    insert_tag['id'] = element.attrib['id']
                    insert_tag['value'] = child.attrib['v']
                    insert_tag['key'] = child.attrib['k']
                    insert_tag['type'] = "regular"
                    tags.append(insert_tag)    
            
        return {'way': way_attribs, 'way_nodes': way_nodes, 'way_tags': tags}


In [ ]:


def get_element(osm_file, tags=('node', 'way', 'relation')):
    """Yield element if it is the right type of tag"""

    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
            root.clear()
            

def validate_element(element, validator, schema=SCHEMA):
    """Raise ValidationError if element does not match schema"""
    if validator.validate(element, schema) is not True:
        field, errors = next(validator.errors.iteritems())
        message_string = "\nElement of type '{0}' has the following errors:\n{1}"
        error_string = pprint.pformat(errors)
        
        raise Exception(message_string.format(field, error_string))

class UnicodeDictWriter(csv.DictWriter, object):
    """Extend csv.DictWriter to handle Unicode input"""

    def writerow(self, row):
        super(UnicodeDictWriter, self).writerow({
            k: (v.encode('utf-8') if isinstance(v, unicode) else v) for k, v in row.iteritems()
        })

    def writerows(self, rows):
        for row in rows:
            self.writerow(row)

# ================================================== #
#               Main Function                        #
# ================================================== #
def process_map(file_in, validate):
    """Iteratively process each XML element and write to csv(s)"""

    with codecs.open(NODES_PATH, 'w') as nodes_file, \
         codecs.open(NODE_TAGS_PATH, 'w') as nodes_tags_file, \
         codecs.open(WAYS_PATH, 'w') as ways_file, \
         codecs.open(WAY_NODES_PATH, 'w') as way_nodes_file, \
         codecs.open(WAY_TAGS_PATH, 'w') as way_tags_file:

        nodes_writer = UnicodeDictWriter(nodes_file, NODE_FIELDS)
        node_tags_writer = UnicodeDictWriter(nodes_tags_file, NODE_TAGS_FIELDS)
        ways_writer = UnicodeDictWriter(ways_file, WAY_FIELDS)
        way_nodes_writer = UnicodeDictWriter(way_nodes_file, WAY_NODES_FIELDS)
        way_tags_writer = UnicodeDictWriter(way_tags_file, WAY_TAGS_FIELDS)

        nodes_writer.writeheader()
        node_tags_writer.writeheader()
        ways_writer.writeheader()
        way_nodes_writer.writeheader()
        way_tags_writer.writeheader()

        validator = cerberus.Validator()

        for element in get_element(file_in, tags=('node', 'way')):
            el = shape_element(element)
            if el:
                if validate is True:
                    validate_element(el, validator)
                
                if element.tag == 'way':
                    ways_writer.writerow(el['way'])
                    way_nodes_writer.writerows(el['way_nodes'])
                    way_tags_writer.writerows(el['way_tags'])

                if element.tag == 'node':
                    nodes_writer.writerow(el['node'])
                    node_tags_writer.writerows(el['node_tags'])
    


if __name__ == '__main__':
    # Note: Validation is ~ 10X slower. For the project consider using a small
    # sample of the map when validating.
    process_map(OSM_PATH, validate=True)

## Creating Tables in SQLITE3 Using Command Line
As my CSV files were populated, I used the command line functionality to create tables for each. The code was as follows

.mode csv
.import nodes.csv nodes
.import nodes_tags.csv nodes_tags
.import ways_nodes.csv ways_nodes
.import ways_tags.csv ways_tags
.imports ways.csv ways


## Ensuring Files were imported properly
We already have an indication of how many nodes and ways there should be in the files from our count types function, so we will query the new tables within the database to ensure the numbers equal one another. 

In [2]:
import sqlite3

In [3]:
sqlite_database = 'lisbon_portugal.db'

In [4]:
conn = sqlite3.connect(sqlite_database)

In [5]:
c = conn.cursor()

In [107]:
QUERY = '''
SELECT COUNT(*) as rows
FROM nodes'''

c.execute(QUERY)
data = c.fetchall()
print data

[(2007823,)]


In [108]:
QUERY = '''
SELECT COUNT(*) as rows
FROM ways'''

c.execute(QUERY)
data = c.fetchall()
print data

[(265109,)]


### Results
The total number of rows from the nodes table (2,007,823) and the total number of ways (265,109) match the figures originally calculated in the count_tags function. This means our data was uploaded into the SQL tables successfully, so we can start querying with confidence. 

## Finding our top users
The following Query_Top_Users will determine the top 10 contributors to the nodes table. 

In [109]:
QUERY_TOP_USERS = '''
SELECT nodes.user, COUNT(*)
FROM nodes
GROUP BY nodes.user
ORDER BY COUNT(*) DESC
LIMIT 10;
'''

c.execute(QUERY_TOP_USERS)
all_data = c.fetchall()
print(all_data)

[(u'Reino Baptista', 176139), (u'topolusitania', 171605), (u'Josef K', 103243), (u'DOSM24', 68350), (u'jsimoes', 68119), (u'ViriatoLusitano_import', 61177), (u'bitormarques', 59789), (u'Lobinho', 57320), (u'ddtuga', 55752), (u'ftavares87', 54060)]


### Results
It appears Reino Baptisa is the most highest contributor to the nodes table with 176,139 entries. Let's see how has contributed most to the ways table

## Highest Contributor, Ways

In [93]:
QUERY_TOP_USERS_WAYS = '''
SELECT ways.user, COUNT(*) as total
FROM ways
GROUP BY ways.user
ORDER BY total desc
LIMIT 20;'''

c.execute(QUERY_TOP_USERS_WAYS)
data = c.fetchall()
print data

[(u'Josef K', 28742), (u'DOSM24', 15494), (u'Reino Baptista', 13896), (u'topolusitania', 12119), (u'Fernando Jorge', 9659), (u'oha', 7527), (u'!i!', 7358), (u'imivorn', 5828), (u'bitormarques', 5354), (u'ddtuga', 5273), (u'rtafav2', 4255), (u'biscas', 4114), (u'Portugaltg', 3675), (u'Lobinho', 3601), (u'ftavares87', 3542), (u'zermes', 3360), (u'mihaiile', 3320), (u'Rafael Marques', 3030), (u'beweta', 2916), (u'Rui Oliveira', 2865)]


Josef K was the highest contributor to the ways data. Reino Baptista, the highest contributor to the nodes table, is actually the third highest in the ways data.

## Examing most active K values in Nodes
We will be running a query to examine the most prevalent k_values in the nodes_tags table. We will be using results from this query to determine what to further examine

In [92]:
QUERY_K_MOST_VALUES = '''
SELECT nodes_tags.key, COUNT(*) as total
FROM nodes_tags
GROUP BY nodes_tags.key
ORDER BY total desc
LIMIT 100;
'''
c.execute(QUERY_K_MOST_VALUES)
data = c.fetchall()
print data

[(u'highway', 15900), (u'name', 15598), (u'amenity', 11677), (u'crossing', 7257), (u'street', 6496), (u'housenumber', 6146), (u'created_by', 5749), (u'natural', 5536), (u'crossing_ref', 5160), (u'power', 4797), (u'city', 4653), (u'shop', 3713), (u'postcode', 3321), (u'source', 2643), (u'country', 2357), (u'ref', 2350), (u'barrier', 1820), (u'phone', 1769), (u'place', 1752), (u'operator', 1684), (u'public_transport', 1588), (u'man_made', 1552), (u'website', 1520), (u'opening_hours', 1341), (u'tourism', 1334), (u'railway', 1093), (u'cuisine', 1056), (u'entrance', 1049), (u'bus', 906), (u'ele', 835), (u'email', 807), (u'type', 792), (u'shelter', 752), (u'supervised', 654), (u'bench', 646), (u'network', 639), (u'wheelchair', 633), (u'bicycle', 556), (u'pt', 545), (u'leisure', 534), (u'historic', 500), (u'foot', 472), (u'access', 460), (u'route_ref', 432), (u'bin', 393), (u'short_name', 377), (u'noexit', 363), (u'internet_access', 349), (u'dispensing', 321), (u'office', 321), (u'facebook', 

### Results
It appears highway, name, amenity, crossing, and street are the most prevalent of the k_values in the dataset. There are a few areas that may be interesting to further examine. Thus, I will attempt to answer the following questions. 

What highway is tagged the most in the nodes dataset? 
What amentities does Lisbon offer the most of?
Is there a type of cuisine that is most prevalent in Lisbon?
What is the most popular religion based on node tags data? 

## What highway is most tagged in the nodes dataset? 

In [6]:
QUERY_HIGHWAY = '''
SELECT nodes_tags.key, nodes_tags.value, COUNT (nodes_tags.value) as total
FROM nodes_tags
WHERE nodes_tags.key = "highway"
GROUP BY nodes_tags.value
ORDER BY total desc
LIMIT 10;'''

c.execute(QUERY_HIGHWAY)
data = c.fetchall()
print data

[(u'highway', u'crossing', 9453), (u'highway', u'bus_stop', 2562), (u'highway', u'traffic_signals', 1859), (u'highway', u'turning_circle', 926), (u'highway', u'street_lamp', 338), (u'highway', u'motorway_junction', 299), (u'highway', u'mini_roundabout', 124), (u'highway', u'give_way', 111), (u'highway', u'stop', 106), (u'highway', u'speed_camera', 46)]


### Results
It appears users tag 'crossings' and 'bus_stop' most often in the data. There seems to be a great deal of bus stops, and the prevalence of crossings may mean the city is built for public transit or pedestrian traffic. 

## What type of amenities does Lisbon offer? 

In [56]:
QUERY_AMENITY = '''
SELECT nodes_tags.key, nodes_tags.value, COUNT (nodes_tags.value) as total
FROM nodes_tags
WHERE nodes_tags.key = "amenity"
GROUP BY nodes_tags.value
ORDER BY total desc
LIMIT 10;'''

c.execute(QUERY_AMENITY)
data = c.fetchall()
print data

[(u'amenity', u'restaurant', 2002), (u'amenity', u'cafe', 1637), (u'amenity', u'parking', 1047), (u'amenity', u'pharmacy', 642), (u'amenity', u'recycling', 580), (u'amenity', u'bench', 541), (u'amenity', u'bank', 486), (u'amenity', u'atm', 388), (u'amenity', u'bar', 342), (u'amenity', u'fuel', 325)]


### Results 
Lisbon, Portugal dataset in which amenity was the key showed restaurants and cafes making up the two most offered amenities in the City. Interestingly, "recycling" was the fifth most tagged amenity, perhaps showing Lisbon's penchant for recycling and environmental conservation

## Is there a type of cuisine that is most prevalent in Lisbon?

In [59]:
QUERY_CUISINE = '''
SELECT nodes_tags.key, nodes_tags.value, COUNT(nodes_tags.value) as total
FROM nodes_tags
WHERE nodes_tags.key = "cuisine"
GROUP BY nodes_tags.value
ORDER BY total desc
LIMIT 10; 
'''
c.execute(QUERY_CUISINE)
data = c.fetchall()
print data

[(u'cuisine', u'regional', 297), (u'cuisine', u'coffee_shop', 83), (u'cuisine', u'burger', 74), (u'cuisine', u'pizza', 63), (u'cuisine', u'portuguese', 59), (u'cuisine', u'italian', 35), (u'cuisine', u'indian', 28), (u'cuisine', u'seafood', 24), (u'cuisine', u'chinese', 23), (u'cuisine', u'chicken', 22)]


### Results
In looking at the data, we see that "regional" is the most tagged type of cuisine, followed by "coffee shop" and "burger". Though this isn't a surprise, a potential issue arises from the data. 

#### Potential Problem
The fifth most popularly tagged cuisine was "portuguese". This isn't different from traditional, or potentially is, that difference isn't clear. It may be beneficial to simply combine this into one category before exporting to CSV for future use. 

## Most K values in Ways

In [7]:
QUERY_K_MOST_VALUES = '''
SELECT ways_tags.key, COUNT(*) as total
FROM ways_tags
GROUP BY ways_tags.key
ORDER BY total desc
LIMIT 20;
'''
c.execute(QUERY_K_MOST_VALUES)
data = c.fetchall()
print data

[(u'highway', 125532), (u'building', 93976), (u'name', 43054), (u'oneway', 36059), (u'source', 18176), (u'surface', 13978), (u'street', 12462), (u'lanes', 11667), (u'landuse', 11198), (u'housenumber', 10859), (u'city', 9327), (u'barrier', 8729), (u'ref', 7045), (u'amenity', 6681), (u'service', 6406), (u'maxspeed', 5740), (u'postcode', 5673), (u'leisure', 5648), (u'country', 5109), (u'bicycle', 4972)]


## What is the most popular religion based on ways tags data? 

In [98]:
QUERY_RELIGION = '''
SELECT ways_tags.key, ways_tags.value, COUNT(ways_tags.value) as total
FROM ways_tags
WHERE ways_tags.key = "religion"
GROUP BY ways_tags.value
ORDER BY total desc
LIMIT 100; 
'''
c.execute(QUERY_RELIGION)
data = c.fetchall()
print data

[(u'religion', u'christian', 337), (u'religion', u'muslim', 3), (u'religion', u'hindu', 2), (u'religion', u'buddhist', 1), (u'religion', u'jewish', 1), (u'religion', u'spiritualist', 1)]


### Results
After examining the ways_tags data for "religion" tags, no surprise, Christian remains the most popular religion in Lisbon. 

## Most Popular City in the dataset

In [97]:
QUERY = '''
SELECT union_tags.key, union_tags.value, COUNT(*) as count 
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) union_tags 
WHERE union_tags.key = 'city' or union_tags.key = "ciudad"
GROUP BY union_tags.value
ORDER BY count desc
LIMIT 10;
'''

c.execute(QUERY)
data = c.fetchall()
print data

[(u'city', u'Lisboa', 7453), (u'city', u'MEM MARTINS', 2627), (u'city', u'Montijo', 1197), (u'city', u'Mem Martins', 282), (u'city', u'Miratejo', 252), (u'city', u'Sintra', 190), (u'city', u'Lisbon', 142), (u'city', u'Alcochete', 121), (u'city', u'Alcabideche', 117), (u'city', u'Lisbonne', 89)]


### Results
The results of the query show Lisboa to be the most popularly tagged City, though there is a potential issue with the results. 

#### Potential Problem
Though Lisboa is the most tagged city in the nodes and ways categories when grouped, we see that 'Lisbon' and 'Lisbonne' also make up a substantial amount of the total tags. These results may all mean the same thing - which may have been best fixed in the previous data set

## Examining Waterways
Lisbon is known for its beaches and position next to the Tagus River. It would be interesting to examine what types of tags.value are associated with the key 'waterway'. 

In [101]:
QUERY_WATERWAYS = '''
SELECT union_tags.key, union_tags.value, COUNT(*) as total
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) union_tags
WHERE union_tags.key = "waterway"
GROUP BY union_tags.value
ORDER BY total desc
LIMIT 100;
'''
c.execute(QUERY_WATERWAYS)
data = c.fetchall()
print data

[(u'waterway', u'stream', 1118), (u'waterway', u'drain', 720), (u'waterway', u'canal', 475), (u'waterway', u'river', 142), (u'waterway', u'ditch', 112), (u'waterway', u'riverbank', 73), (u'waterway', u'reservoir', 29), (u'waterway', u'dock', 21), (u'waterway', u'waterfall', 17), (u'waterway', u'weir', 16), (u'waterway', u'dam', 10), (u'waterway', u'lock_gate', 6), (u'waterway', u'boatyard', 2), (u'waterway', u'fuel', 1), (u'waterway', u'lake', 1), (u'waterway', u'turning_point', 1)]


### Results
It appears that "stream" is the most tagged value for "waterway" in the joined dataset with 1118 tags. 

#### Potential Problem
The fourth most tagged waterway was 'river'. This is not a surprise considering Lisbon is positioned next to the Tagus River, however, 'river' may be perceived as the same value as 'stream' which is the most tagged 'waterway' in the dataset. These may be the same actual bodies, they may be different. Human-entered data doesn't lend itself to standardization. 

## Keeping Country First
We would like to determine if people have tagged different countries, though all are in Portugal. 

In [100]:
QUERY_COUNTRY = '''
SELECT union_tags.key, union_tags.value, COUNT(*) as total
FROM (SELECT * FROM nodes_tags UNION ALL SELECT * FROM ways_tags) union_tags
WHERE union_tags.key = "country"
GROUP BY union_tags.value
ORDER BY total desc
LIMIT 100;
'''
c.execute(QUERY_COUNTRY)
data = c.fetchall()
print data

[(u'country', u'PT', 7430), (u'country', u'Portugal', 6), (u'country', u'ES', 5), (u'country', u'BR', 2), (u'country', u'CN', 2), (u'country', u'DE', 2), (u'country', u'NL', 2), (u'country', u'SE', 2), (u'country', u'US', 2), (u'country', u'BG', 1), (u'country', u'CZ', 1), (u'country', u'DK', 1), (u'country', u'FR', 1), (u'country', u'GB', 1), (u'country', u'IN', 1), (u'country', u'IT', 1), (u'country', u'LU', 1), (u'country', u'LV', 1), (u'country', u'PH', 1), (u'country', u'RO', 1), (u'country', u'RU', 1), (u'country', u'UA', 1)]


### Results
Where 'country' tags are used in the data, I was surprised to see the abbreviation 'PT' used more often than 'Portugal'. __Suggestion__: There is no difference in the two values 'Portugal' and 'PT' as they represent the same thing, thus 'Portugal' should be changed into 'PT' before uploading into CSV. 

# Conclusion

### Filesizes: 
lisbon_portugal.osm: 436.1 MB
nodes.csv: 167.9 MB
nodes_tags.csv: 5.7 MB
ways.csv: 15.9 MB
ways_tags.csv: 18.9 MB
ways_nodes.csv: 61.1 MB

### Potential Problems Overview:

__Food Issues:__ The fifth most popularly tagged cuisine was "portuguese". This isn't different from "traditional", which was the most tagged type of cuisine in the dataset, or potentially is, that difference isn't clear. It may be beneficial to simply combine this into one category before exporting to CSV for future use. 

__Inconsistent City Names__: As indicated previously, Lisboa was the most tagged City name in the dataset. However, there were other City tags that were very close in intent and spelling ("Lisbon", "Lisbonne"). __Suggestion__: This inconsistency may be grouped together before exporting to CSV, however, the inherent danger is that we don't understand which of the towns or neighborhoods could also be consolidated into "Lisboa". This may be a larger issue than the researcher must dive into.  

__Waterways Inconsistent__: The most tagged 'waterway' was 'stream', though 'river' was a close fourth. There is no perceived justification why a 'stream' is different from a 'river', thus this inconsistency in naming convention may just be because of human error. __Suggestion__: The researcher can change "river" to "stream" with a function prior to writing to a CSV, however, the researcher will have to make the decision that there can be no rivers that aren't streams. 

__Country Tags__: Where 'country' tags are used in the data, I was surprised to see the abbreviation 'PT' used more often than 'Portugal'. __Suggestion__: There is no difference in the two values 'Portugal' and 'PT' as they represent the same thing, thus 'Portugal' should be changed into 'PT' before uploading into CSV. 